In [1]:
import pandas as pd
import json
import re
import pymongo
import jieba
import operator
import numpy as np

In [2]:
#連接 mongodb

client = pymongo.MongoClient(host='123.241.175.34', port=27017)

In [3]:
#mongodb帳號密碼

client.admin.authenticate('root', '1qaz@WSX3edc')








True

In [4]:
#設定名稱為test的資料庫 也可用 db = client['test']
client.database_names()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  


['ETL', 'admin', 'config', 'local', 'raw_article', 'test']

In [5]:
db_ETL=client.ETL

In [6]:
#顯示db中所有collections
db_ETL.list_collection_names()

['article_web_seg', 'article_news_seg', 'article_social_seg']

In [7]:
article_social=db_ETL.article_social_seg.find()
article_web=db_ETL.article_web_seg.find()
article_news=db_ETL.article_news_seg.find()

In [8]:
df_article_social=pd.DataFrame(list(article_social))
df_article_web=pd.DataFrame(list(article_web))
df_article_news=pd.DataFrame(list(article_news))

In [9]:
article_social_seg=df_article_social['seg']
article_web_seg=df_article_web['seg']
article_news_seg=df_article_news['seg']

In [10]:
article_news_seg[:5]

0    國內 刷卡 回饋 重 洗牌 　 無腦 刷 排行榜 大車 拼 \n \n \n 　 　 下半年...
1    直擊雙 11 電商 AI 倉儲 　 下 單到 出貨僅 10 分鐘 \n \n \n 　 　 ...
2    雙 11 網購 主打 賓士車 　 交易 攻略 回饋 16.8 萬元 \n \n \n 　 　...
3    刷 信用卡 繳納地 價稅 　 24 家銀行 免收 手續費 \n \n \n 　 　 今 20...
4    4 張 行動支付 聯名卡 對決 　 街口 卡力尬 Pi 拍 錢包 \n \n \n 　 　 ...
Name: seg, dtype: object

In [11]:
bank={'中信':'中信','中國信託':'中信' ,'中信金':'中信' ,'中信金控':'中信','中信銀':'中信',
'玉山':'玉山','玉山金':'玉山','玉山金控':'玉山','玉山銀':'玉山',
'Richart':'Richart', 'richart':'Richart',
'KOKO':'KOKO','koko':'KOKO',
'國泰':'國泰','國泰世華':'國泰','國泰金':'國泰','國泰金控':'國泰',
'台新':'台新','台新金':'台新','台新金控':'台新','台新銀':'台新','台新銀行':'台新',
'永豐':'永豐','永豐銀':'永豐','永豐金':'永豐','永豐金控':'永豐','永豐銀行':'永豐',
'兆豐':'兆豐','兆豐銀':'兆豐','兆豐金':'兆豐','兆豐金控':'兆豐','兆豐銀行':'兆豐',
'花旗':'花旗','花旗銀':'花旗','花旗銀行':'花旗',
'一銀':'一銀','第一銀':'一銀','第一金':'一銀','第一金控':'一銀','第一銀行':'一銀',
'富邦':'富邦','富邦金':'富邦','富邦金控':'富邦','富邦銀':'富邦','富邦銀行':'富邦',
'彰化':'彰化','彰化銀':'彰化','彰化金控':'彰化','彰化金':'彰化','彰化銀行':'彰化',
'新光':'新光','新光銀':'新光','新光金':'新光','新光金控':'新光','新光銀行':'新光',
'匯豐':'匯豐','匯豐銀行':'匯豐','匯豐銀':'匯豐',
'陽信銀行':'陽信','陽信':'陽信','陽信銀':'陽信',
'元大':'元大','元大銀行':'元大','元大金控':'元大','元大銀':'元大',
'星展':'星展','星展銀行':'星展','星展銀':'星展',
'渣打':'渣打','渣打銀行':'渣打','渣打銀':'渣打',
'上海商銀':'上海商銀','上海銀行':'上海商銀',
'華南':'華南','華南銀行':'華南','華南金控':'華南','華南銀':'華南',
'凱基':'凱基','凱基銀行':'凱基','凱基金控':'凱基',
'合作金庫':'合作金庫','合庫':'合作金庫',
'王道':'王道','王道銀行':'王道',
'澳盛':'澳盛','澳盛銀行':'澳盛',
'遠東商銀':'遠東商銀','遠銀':'遠東商銀','遠東銀':'遠東商銀',
'土地銀行':'土地','土銀':'土地',
'聯邦銀行':'聯邦','聯邦':'聯邦',
'台灣銀行':'臺灣銀行','臺灣銀行':'臺灣銀行',
'京城':'京城','京城銀行':'京城',
'日盛':'日盛','日盛銀行':'日盛',
'華泰':'華泰','華泰銀行':'華泰',
'台中銀行':'台中銀行','台中銀':'台中銀行',
'高雄銀行':'高雄銀行','高雄銀':'高雄銀行',
'台灣企銀':'臺灣企銀','臺灣企銀':'臺灣企銀','臺企銀':'臺灣企銀',
'瑞興銀行':'瑞興銀行','瑞興':'瑞興銀行',
}


card={
'linepay':'LINEPay','Linepay':'LINEPay','LinePay':'LINEPay','LINEPAY':'LINEPay','LINE PAY':'LINEPay','Line pay':'LINEPay',
'line pay':'LINEPay',
'GOGO':'GoGo','gogo':'GoGo','黑狗':'GoGo','黑狗卡':'GoGo','gogo卡':'GoGo','GOGO卡':'GoGo','Gogo卡':'GoGo','Gogo':'GoGo','GoGo':'GoGo',
'flygo':'Flygo','Flygo':'Flygo','FLYGO':'Flygo','飛狗':'Flygo','飛狗卡':'Flygo',
'大戶':'dawho','dawho':'dawho','Dawho':'dawho',
'PI':'Pi','Pi':'Pi','pi卡':'Pi','PI卡':'Pi',
'ubear':'Ubear','UBEAR':'Ubear','Ubear':'Ubear','ubear':'Ubear','u bear':'Ubear','U bear':'Ubear','u bear':'Ubear', 'Ubear卡':'Ubear','U Bear':'Ubear',
'only':'Only','ONLY':'Only','Only':'Only',
'賴點':'賴點卡','賴點卡':'賴點卡', 
'酷玩卡':'酷玩卡','酷玩卡':'酷玩卡',
'饗樂':'饗樂卡','饗樂卡':'饗樂卡',
'koko':'KOKO','KOKO':'KOKO','KoKo':'KOKO',
'星燦':'星燦卡','星燦卡':'星燦卡',
'統一獅':'統一獅聯名卡',
'中信兄弟':'中信兄弟聯名卡',
'富邦悍將':'富邦悍將聯名卡',
'My樂現金回饋卡':'My樂現金回饋卡','My樂現金回饋':'My樂現金回饋卡',
'哆啦A夢JCB聯名卡':'哆啦A夢JCB聯名卡','哆啦A夢':'哆啦A夢JCB聯名卡',
'Omiyage':'Omiyage','omiyage':'Omiyage',
'尊御世界卡':'尊御世界卡',
'鑽金一卡通聯名卡':'鑽金','鑽金':'鑽金',
'炫晶御璽卡':'炫晶御璽卡','炫晶卡':'炫晶御璽卡','炫晶':'炫晶御璽卡',
'現金回饋御璽卡':'現金回饋御璽卡',
'PChome Prime聯名卡':'PChome Prime聯名卡','Prime聯名卡':'PChome Prime聯名卡',
'蝦皮購物聯名卡':'蝦皮購物聯名卡',
'Lamigo聯名卡':'Lamigo聯名卡','Lamigo':'Lamigo聯名卡','lamigo':'Lamigo聯名卡',
'SPORT卡':'SPORT卡','SPORT':'SPORT卡',
'e秒刷鈦金卡':'e秒刷鈦金卡','e秒刷':'e秒刷鈦金卡',
'小小兵':'小小兵','小小兵':'小小兵信用卡',
'everyday鈦金卡':'everyday鈦金卡','everyday':'everyday鈦金卡',
'街口聯名卡':'街口聯名卡',
'Gogoro聯名卡':'Gogoro聯名卡','Gogoro':'Gogoro聯名卡','gogoro':'Gogoro聯名卡',
'現金回饋御璽卡':'現金回饋御璽卡',
}

behavior={
'跑步':'運動','健身房':'運動','運動':'運動',
'旅遊':'旅遊','旅行':'旅遊','機票':'旅遊','飯店':'旅遊','機場':'旅遊','出國':'旅遊',
'保險':'保險','保費':'保險',
'加油':'加油','中油':'加油',
'國外':'國外','海外':'國外',
'電影':'電影','國賓':'電影','威秀':'電影',
'網購':'網購' ,'網拍':'網購', '蝦皮':'網購', '電商':'網購','pchome':'網購','PChome':'網購','Yahoo':'網購','momo':'網購','MOMO':'網購',
'行動支付':'行動支付','PI錢包':'行動支付','pi錢包':'行動支付','街口':'行動支付','Pi錢包':'行動支付',
'新光三越':'百貨','SOGO':'百貨','sogo':'百貨','遠百':'百貨','遠東百貨':'百貨',
'紅利點數':'紅利','紅利兌換':'紅利',
'超商':'超商','7-11':'超商','全家':'超商','全家便利':'超商','萊爾富':'超商','OK便利':'超商',
'屈臣氏':'藥妝','康是美':'藥妝',
'大潤發':'量販店','家樂福':'量販店','愛買':'量販店',
'全聯':'超市','頂好':'超市',
}

In [12]:
behavior['跑步']

'運動'

In [13]:
#article_social_seg=df_article_social['seg']
#article_web_seg=df_article_web['seg']
#article_news_seg=df_article_news['seg']
#bank
#news
n_soc=len(article_social_seg)
bank_tag_soc=[]
for article in range(n_soc):
    article_bank=set()
    for word in article_social_seg[article].split(' '):
            if word in bank:
                article_bank.add(bank[word])
    bank_tag_soc.append(list(article_bank))
print('soc')
print(bank_tag_soc[:10])

#web
n_web=len(article_web_seg)
bank_tag_web=[]
for article in range(n_web):
    article_bank=set()
    for word in article_web_seg[article].split(' '):
            if word in bank:
                article_bank.add(bank[word])
    bank_tag_web.append(list(article_bank))
print('web')
print(bank_tag_web[:10])


#social
n_news=len(article_news_seg)
bank_tag_news=[]
for article in range(n_news):
    article_bank=set()
    for word in article_news_seg[article].split(' '):
            if word in bank:
                article_bank.add(bank[word])
    bank_tag_news.append(list(article_bank))
print('news')
print(bank_tag_news[:10])

soc
[[], [], ['台新'], ['永豐', '臺灣銀行'], [], ['元大', '富邦', '玉山', '台新'], ['台新'], ['永豐'], ['Richart'], []]
web
[['玉山'], ['國泰'], ['Richart', '匯豐', '永豐', '華南', '上海商銀', '台新', '中信', '星展'], ['凱基', '聯邦', '花旗', '兆豐', '中信', '星展', '永豐', '上海商銀', '彰化', '富邦', '匯豐', '澳盛', '玉山', '華南', '台新', '元大', '新光', '一銀', '日盛', '陽信', '國泰'], [], [], [], ['遠東商銀', '富邦', '凱基', '永豐', '台中銀行', '合作金庫', '聯邦', '玉山', '花旗', '兆豐', '國泰', '新光', '台新', '元大', '中信', '一銀'], ['凱基', '聯邦', '花旗', '兆豐', '中信', '永豐', '彰化', '渣打', '富邦', '台中銀行', '玉山', '華南', '台新', '元大', '新光', '日盛', '合作金庫', '陽信', '國泰'], ['渣打', '富邦', 'Richart', '匯豐', '永豐', '上海商銀', 'KOKO', '澳盛', '玉山', '花旗', '兆豐', '國泰', '台新', '元大', '中信', '星展']]
news
[['渣打', '富邦', '永豐', 'KOKO', '聯邦', '玉山', '國泰', '中信'], [], ['玉山', '台新'], ['遠東商銀', '凱基', '臺灣企銀', '聯邦', '花旗', '兆豐', '中信', '星展', '華泰', '永豐', '土地', '高雄銀行', '渣打', '富邦', '台中銀行', '玉山', '台新', '華南', '元大', '新光', '一銀', '日盛', '合作金庫', '陽信', '國泰'], ['兆豐', '中信', '玉山'], ['富邦', '永豐', '聯邦', '玉山', '國泰', '台新', '中信', '一銀'], ['渣打', '富邦', '華南', '聯邦', '玉山', '花旗', '國泰'

In [14]:
#card
#soc

n_soc=len(article_social_seg)
card_tag_soc=[]
for article in range(n_soc):
    article_card=set()
    for word in article_social_seg[article].split(' '):
            if word in card:
                article_card.add(card[word])
    card_tag_soc.append(list(article_card))
print('soc')
print(card_tag_soc[:10])

#web
n_web=len(article_web_seg)
card_tag_web=[]
for article in range(n_web):
    article_card=set()
    for word in article_web_seg[article].split(' '):
            if word in card:
                article_card.add(card[word])
    card_tag_web.append(list(article_card))
print('web')
print(card_tag_web[:10])


#news
n_news=len(article_news_seg)
card_tag_news=[]
for article in range(n_news):
    article_card=set()
    for word in article_news_seg[article].split(' '):
            if word in card:
                article_card.add(card[word])
    card_tag_news.append(list(article_card))
print('news')
print(card_tag_news[:10])

soc
[[], [], [], ['dawho'], ['dawho'], [], ['GoGo'], ['dawho'], ['LINEPay'], ['小小兵信用卡']]
web
[['Pi'], [], ['小小兵信用卡', 'dawho', '炫晶御璽卡'], [], [], [], [], [], ['尊御世界卡'], ['GoGo', '炫晶御璽卡', 'Pi', 'Ubear', 'KOKO', '現金回饋御璽卡', '鑽金', 'e秒刷鈦金卡']]
news
[['小小兵信用卡', 'Pi', 'KOKO', '現金回饋御璽卡', 'dawho', '賴點卡', 'Only'], [], ['Pi'], [], ['Pi'], [], ['Only', '蝦皮購物聯名卡', '賴點卡', 'My樂現金回饋卡'], ['Only', 'SPORT卡'], [], []]


In [15]:
#behavior      
#soc

n_soc=len(article_social_seg)
behavior_tag_soc=[]
for article in range(n_soc):
    article_behavior=set()
    for word in article_social_seg[article].split(' '):
            if word in behavior:
                article_behavior.add(behavior[word])
    behavior_tag_soc.append(list(article_behavior))
print('soc')
print(behavior_tag_soc[:10])

#web
n_web=len(article_web_seg)
behavior_tag_web=[]
for article in range(n_web):
    article_behavior=set()
    for word in article_web_seg[article].split(' '):
            if word in behavior:
                article_behavior.add(behavior[word])
    behavior_tag_web.append(list(article_behavior))
print('web')
print(behavior_tag_web[:10])


#news
n_news=len(article_news_seg)
behavior_tag_news=[]
for article in range(n_news):
    article_behavior=set()
    for word in article_news_seg[article].split(' '):
            if word in behavior:
                article_behavior.add(behavior[word])
    behavior_tag_news.append(list(article_behavior))
print('news')
print(behavior_tag_news[:10])

soc
[[], [], ['行動支付'], ['旅遊'], [], [], [], [], [], ['量販店', '藥妝']]
web
[['旅遊', '行動支付', '超商', '國外'], ['旅遊'], ['網購', '旅遊', '國外'], ['旅遊', '國外'], [], ['旅遊'], ['保險', '紅利'], ['加油', '行動支付'], ['旅遊', '國外'], ['保險', '超商', '加油', '電影', '網購', '旅遊', '國外']]
news
[['超商', '電影', '網購', '行動支付', '旅遊'], ['網購'], ['網購', '保險'], ['行動支付'], ['網購', '行動支付', '國外', '超商'], ['量販店', '超市', '行動支付', '超商'], ['行動支付', '旅遊', '國外'], ['運動', '國外'], [], ['超市', '行動支付']]


In [16]:
#合併dataframe
#soc
d_soc={'bank_tag':bank_tag_soc,'card_tag':card_tag_soc,'behavior_tag':behavior_tag_soc}
df_tag_soc=pd.DataFrame(data=d_soc)


#web
d_web={'bank_tag':bank_tag_web,'card_tag':card_tag_web,'behavior_tag':behavior_tag_web}
df_tag_web=pd.DataFrame(data=d_web)


#news
d_news={'bank_tag':bank_tag_news,'card_tag':card_tag_news,'behavior_tag':behavior_tag_news}
df_tag_news=pd.DataFrame(data=d_news)


In [17]:
#df_tag_soc[:10]
#df_tag_web[:10]
df_tag_news[:10]

,bank_tag,card_tag,behavior_tag
0,"[渣打, 富邦, 永豐, KOKO, 聯邦, 玉山, 國泰, 中信]","[小小兵信用卡, Pi, KOKO, 現金回饋御璽卡, dawho, 賴點卡, Only]","[超商, 電影, 網購, 行動支付, 旅遊]"
1,[],[],[網購]
2,"[玉山, 台新]",[Pi],"[網購, 保險]"
3,"[遠東商銀, 凱基, 臺灣企銀, 聯邦, 花旗, 兆豐, 中信, 星展, 華泰, 永豐, 土...",[],[行動支付]
4,"[兆豐, 中信, 玉山]",[Pi],"[網購, 行動支付, 國外, 超商]"
5,"[富邦, 永豐, 聯邦, 玉山, 國泰, 台新, 中信, 一銀]",[],"[量販店, 超市, 行動支付, 超商]"
6,"[渣打, 富邦, 華南, 聯邦, 玉山, 花旗, 國泰, 台新, 元大, 中信]","[Only, 蝦皮購物聯名卡, 賴點卡, My樂現金回饋卡]","[行動支付, 旅遊, 國外]"
7,"[玉山, 永豐]","[Only, SPORT卡]","[運動, 國外]"
8,"[Richart, 富邦, 中信, 台新]",[],[]
9,"[華泰, 富邦, 永豐, 聯邦, 玉山, 國泰, 台新, 中信, 一銀]",[],"[超市, 行動支付]"


In [18]:
#計算card behavior出現次數矩陣
all_bank=set([bank[b] for b in bank])
all_card=set([card[c] for c in card])
all_behavior=set([behavior[b] for b in behavior])


#建立df並填0
df_card_behavoir=pd.DataFrame(index=all_card,columns=all_behavior)
for i in df_card_behavoir.columns:
    df_card_behavoir[i]=0
df_card_behavoir

,百貨,保險,運動,超商,量販店,加油,電影,超市,紅利,藥妝,網購,行動支付,旅遊,國外
KOKO,0,0,0,0,0,0,0,0,0,0,0,0,0,0
dawho,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PChome Prime聯名卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0
everyday鈦金卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0
饗樂卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Gogoro聯名卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0
酷玩卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0
e秒刷鈦金卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0
星燦卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0
街口聯名卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
df_card_behavoir_soc=df_card_behavoir
df_card_behavoir_web=df_card_behavoir
df_card_behavoir_news=df_card_behavoir

In [20]:
#計算
#soc

ntag_soc=len(df_tag_soc)
#填入df之value
for i in  range(ntag_soc):
    for c_tag in df_tag_soc['card_tag'][i]:
        for b_tag in df_tag_soc['behavior_tag'][i]:
            df_card_behavoir_soc[b_tag].loc[c_tag]+=1
df_card_behavoir_soc

,百貨,保險,運動,超商,量販店,加油,電影,超市,紅利,藥妝,網購,行動支付,旅遊,國外
KOKO,86,116,19,273,157,149,126,52,24,13,475,266,293,238
dawho,22,37,8,62,21,35,56,24,11,10,94,68,130,117
PChome Prime聯名卡,14,13,0,12,13,14,18,12,5,2,37,14,19,18
everyday鈦金卡,1,2,1,1,1,1,2,2,0,0,3,0,8,7
饗樂卡,49,40,14,43,71,46,167,15,6,5,139,67,135,87
Gogoro聯名卡,16,19,0,17,15,17,15,10,3,3,22,18,24,18
酷玩卡,11,9,4,14,20,21,21,6,6,0,19,15,25,18
e秒刷鈦金卡,27,35,3,44,42,40,38,15,6,8,155,65,80,72
星燦卡,3,4,1,3,3,7,2,2,1,1,10,6,9,1
街口聯名卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
#web

ntag_web=len(df_tag_web)
#填入df之value
for i in  range(ntag_web):
    for c_tag in df_tag_web['card_tag'][i]:
        for b_tag in df_tag_web['behavior_tag'][i]:
            df_card_behavoir_web[b_tag].loc[c_tag]+=1
df_card_behavoir_web

,百貨,保險,運動,超商,量販店,加油,電影,超市,紅利,藥妝,網購,行動支付,旅遊,國外
KOKO,96,151,29,353,169,191,185,77,46,21,560,337,342,294
dawho,23,60,12,89,24,55,82,37,23,11,130,97,178,171
PChome Prime聯名卡,15,14,0,20,13,15,25,12,7,2,49,15,24,20
everyday鈦金卡,1,3,1,2,2,2,6,4,3,0,4,1,12,10
饗樂卡,61,121,22,65,89,135,333,19,34,9,180,266,339,183
Gogoro聯名卡,24,32,4,33,23,22,34,12,14,3,36,32,47,31
酷玩卡,12,10,4,17,21,26,24,8,10,0,25,16,34,22
e秒刷鈦金卡,28,50,4,66,51,58,62,24,13,12,194,90,98,99
星燦卡,4,4,2,6,4,10,4,3,2,1,15,11,10,4
街口聯名卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
#news

ntag_news=len(df_tag_news)
#填入df之value
for i in  range(ntag_news):
    for c_tag in df_tag_news['card_tag'][i]:
        for b_tag in df_tag_news['behavior_tag'][i]:
            df_card_behavoir_news[b_tag].loc[c_tag]+=1
df_card_behavoir_news

,百貨,保險,運動,超商,量販店,加油,電影,超市,紅利,藥妝,網購,行動支付,旅遊,國外
KOKO,98,152,32,367,169,193,190,82,48,24,581,351,355,305
dawho,27,65,14,96,24,55,88,37,25,11,139,105,198,184
PChome Prime聯名卡,15,14,0,20,13,15,26,12,7,2,57,18,24,22
everyday鈦金卡,1,3,1,3,3,2,6,4,3,0,6,3,15,11
饗樂卡,61,122,22,65,89,136,338,19,35,9,183,269,345,184
Gogoro聯名卡,24,33,5,38,24,24,37,13,17,4,41,35,59,35
酷玩卡,12,10,4,17,21,26,25,8,11,0,26,16,36,22
e秒刷鈦金卡,28,50,4,66,51,58,62,24,13,12,195,90,98,99
星燦卡,4,4,2,6,4,10,4,3,2,1,15,11,10,4
街口聯名卡,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
#export
df_card_behavoir_news.to_excel(r'./card_behavoir_news_matrix.xlsx','w',encoding='utf-16')
df_card_behavoir_web.to_excel(r'./card_behavoir_web_matrix.xlsx','w',encoding='utf-16')
df_card_behavoir_soc.to_excel(r'./card_behavior_soc_matrix.xlsx','w',encoding='utf-16')